In [6]:
import sys
import os
package_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, package_root)

In [7]:
from image_acquisition.image_sensor import *
from image_acquisition.playback_images import *
from image_acquisition.video_file_writer import *
#from image_acquisition.ueye_python_driver import UEyePythonDriver

from feature_computation.object_tracking import *
from feature_computation.object_detection_and_tracking import *
from feature_computation.visualization import *

import matplotlib as plt

In [8]:
# Load the ONNX model

onnx_model_path = "../../../../PRISM/Daten/makerspace_20241126/export/onnx/simplified-model.onnx"


print("Absolute Path:", os.path.abspath(onnx_model_path))
video_path = "../../../../PRISM/Daten/makerspace_20241126/videos/rec_1.avi"
print("abs path:", os.path.abspath(video_path))
output_path = "../../PRISM/Daten/WLA16/object_detection/default.avi"


no_classes = 6

Absolute Path: c:\Users\xfebloj\Documents\Projekte\PRISM\Daten\makerspace_20241126\export\onnx\simplified-model.onnx
abs path: c:\Users\xfebloj\Documents\Projekte\PRISM\Daten\makerspace_20241126\videos\rec_1.avi


In [9]:
visualizer = ObjectDetectionVisualization(no_classes)

In [10]:
try:
    with DirectoryImageSensor(directory_path="C:/Users/xfebloj/Documents/Projekte/prism/Daten/makerspace_20241126/videos/rec_3/") as img_src:
    #with VideoFileImageSensor(video_path=video_path) as img_src:
    #with UEyePythonDriver() as img_src:
        with video_file_writer_matching_image_sensor(img_src, output_path) as writer:
            with  ONNXObjectDetector(onnx_model_path) as neural_network:

                def callback(frame:np.ndarray):
                    detections = neural_network.detect(frame)
                    detections =  nms(detections, nms_thr=0.5, det_thr=0.5)
                    
                    disp_img = frame.copy()
                    visualizer.draw_visualization_on_frame(disp_img,detections, show_class_id=True)

                    cv2.imshow("object detection", disp_img)

                    writer.write(disp_img)

                    # Check if 'q' is pressed to quit
                    if cv2.waitKey(-1) & 0xFF == ord('q'):
                        raise NoMoreImages


                run_playback(img_src, callback)
except Exception as e:
    print(e)    

finally:
    cv2.destroyAllWindows()

onnx_session profile:  
